# 1.0 Load Packages & Testing Data

In [1]:
Threads.nthreads()

6

In [2]:
# Load Packages
using Clustering
using ParallelKMeans
using BenchmarkTools
using DelimitedFiles
using mlpack

## Read Data as transposed matrices since Julia is column major

In [3]:
X_1m = permutedims(DelimitedFiles.readdlm("data_1m.csv", ',', Float64));

In [4]:
X_100k = permutedims(DelimitedFiles.readdlm("data_100k.csv", ',', Float64));

In [5]:
X_10k = permutedims(DelimitedFiles.readdlm("data_10k.csv", ',', Float64));

In [6]:
X_1k = permutedims(DelimitedFiles.readdlm("data_1k.csv", ',', Float64));

## Row-wise Data For mlpack

In [7]:
X_1m_mlp = permutedims(X_1m);

In [8]:
X_100k_mlp = permutedims(X_100k);

In [9]:
X_10k_mlp = permutedims(X_10k);

In [10]:
X_1k_mlp = permutedims(X_1k);

# Elbow Method MLPACK

In [11]:
@btime [mlpack.kmeans(i, $X_1m_mlp; algorithm="hamerly", max_iterations=1000)[1] for i = 2:10];

  220.357 s (85 allocations: 2.08 GiB)


In [12]:
@btime [mlpack.kmeans(i, $X_100k_mlp; algorithm="hamerly", max_iterations=1000)[1] for i = 2:10];

  20.936 s (85 allocations: 212.88 MiB)


In [13]:
@btime [mlpack.kmeans(i, $X_10k_mlp; algorithm="hamerly", max_iterations=1000)[1] for i = 2:10];

  824.295 ms (85 allocations: 21.30 MiB)


In [14]:
@btime [mlpack.kmeans(i, $X_1k_mlp; algorithm="hamerly", max_iterations=1000)[1] for i = 2:10];

  26.388 ms (85 allocations: 2.14 MiB)


# 2.0 Elbow Method Clustering.jl

In [15]:
@btime [Clustering.kmeans($X_1m, i; tol=1e-6, maxiter=1000).totalcost for i = 2:10];

  538.531 s (31820 allocations: 33.24 GiB)


In [16]:
@btime [Clustering.kmeans($X_100k, i; tol=1e-6, maxiter=1000).totalcost for i = 2:10];

  33.157 s (20032 allocations: 2.13 GiB)


In [17]:
@btime [Clustering.kmeans($X_10k, i; tol=1e-6, maxiter=1000).totalcost for i = 2:10]; 

  742.375 ms (7614 allocations: 82.82 MiB)


In [18]:
@btime [Clustering.kmeans($X_1k, i; tol=1e-6, maxiter=1000).totalcost for i = 2:10]; 

  17.098 ms (1699 allocations: 2.34 MiB)


# 3.0 Elbow Method Speed ParallelKMeans.jl

## Lloyd

In [19]:
@btime [ParallelKMeans.kmeans(Lloyd(), $X_1m, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  20.554 s (40737 allocations: 210.10 MiB)


In [20]:
@btime [ParallelKMeans.kmeans(Lloyd(), $X_100k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  2.913 s (56992 allocations: 26.27 MiB)


In [21]:
@btime [ParallelKMeans.kmeans(Lloyd(), $X_10k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  175.590 ms (34558 allocations: 5.56 MiB)


In [22]:
@btime [ParallelKMeans.kmeans(Lloyd(), $X_1k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  6.093 ms (10349 allocations: 1.35 MiB)


## Hamerly

In [23]:
@btime [ParallelKMeans.kmeans(Hamerly(), $X_1m, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  11.518 s (62467 allocations: 350.25 MiB)


In [24]:
@btime [ParallelKMeans.kmeans(Hamerly(), $X_100k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  966.373 ms (58405 allocations: 40.85 MiB)


In [25]:
@btime [ParallelKMeans.kmeans(Hamerly(), $X_10k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  99.897 ms (61185 allocations: 10.27 MiB)


In [26]:
@btime [ParallelKMeans.kmeans(Hamerly(), $X_1k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  6.350 ms (16373 allocations: 2.25 MiB)


# Elkan

In [27]:
@btime [ParallelKMeans.kmeans(Elkan(), $X_1m, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  14.019 s (55965 allocations: 701.39 MiB)


In [28]:
@btime [ParallelKMeans.kmeans(Elkan(), $X_100k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  1.131 s (50298 allocations: 75.12 MiB)


In [29]:
@btime [ParallelKMeans.kmeans(Elkan(), $X_10k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  79.120 ms (49220 allocations: 12.43 MiB)


In [30]:
@btime [ParallelKMeans.kmeans(Elkan(), $X_1k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  6.464 ms (16613 allocations: 2.61 MiB)


## YingYang

In [31]:
@btime [ParallelKMeans.kmeans(Yinyang(7), $X_1m, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  9.970 s (23622 allocations: 346.02 MiB)


In [32]:
@btime [ParallelKMeans.kmeans(Yinyang(7), $X_100k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  1.146 s (31409 allocations: 37.89 MiB)


In [33]:
@btime [ParallelKMeans.kmeans(Yinyang(7), $X_10k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  108.337 ms (24498 allocations: 6.24 MiB)


In [34]:
@btime [ParallelKMeans.kmeans(Yinyang(7), $X_1k, i; tol=1e-6, max_iters=1000, verbose=false).totalcost for i = 2:10];

  7.044 ms (9805 allocations: 1.53 MiB)
